# Preprocessing

In [ ]:
import pandas as pd
import numpy as np
import os
import yaml
from tqdm.notebook import trange, tqdm

node = !hostname
if "sc" in node[0]:
    base_path = "/sc-projects/sc-proj-ukb-cvd"
else: base_path = "/data/analysis/ag-reils/ag-reils-shared/cardioRS"
print(base_path)

dataset_name = "211110_anewbeginning"
mapping_path = f"{base_path}/data/mapping"
dataset_path = f"{base_path}/data/2_datasets_pre/{dataset_name}"

In [ ]:
from pathlib import Path
Path(dataset_path).mkdir(parents=True, exist_ok=True)

In [ ]:
import wandb
import pathlib

project="recordgraphs"
entity="cardiors"
artifact_date = "220531"

In [ ]:
artifact_name = "final_records_omop"
type_name = "prepare_records"
description = "Cleaned first patient records from the data portal in long format"

In [ ]:
artifact_path_new = f"{dataset_path}/artifacts/{artifact_name}_{artifact_date}.feather"
records_new = pd.read_feather(artifact_path_new)

In [ ]:
records_new.shape

In [ ]:
records_new.eid.unique()

In [ ]:
artifact_path_old = f"{dataset_path}/artifacts/{artifact_name}_220407.feather"
records_old = pd.read_feather(artifact_path_old)

In [ ]:
records_old.shape

In [ ]:
eid_test = 1000018
records_temp_old = records_old.query("eid ==@eid_test")
records_temp_new = records_new.query("eid ==@eid_test")

In [ ]:
print(records_temp_old.query("vocabulary=='phecode'").shape)
print(records_temp_new.query("vocabulary=='phecode'").shape)

In [ ]:
records_temp_old.query("vocabulary=='phecode'").sort_values("date").query("origin=='hes_icd10'")

In [ ]:
records_temp_new.query("vocabulary=='phecode'").sort_values("date").query("origin=='hes_icd10'")#.query("date>'01-01-2021'")

In [ ]:
records_temp_old.sort_values("date").query("vocabulary=='phecode'")

In [ ]:
records_new.vocabulary.unique()

In [ ]:
vocab_list = ['SNOMED', 'RxNorm', 'CVX']
records_raw = records_new.query("vocabulary==@vocab_list")

In [ ]:
records_raw.query("unique_record_id == 70")

In [ ]:
records_phecodes = records_new.query("vocabulary!=@vocab_list")

In [ ]:
phecode_unique_ids = records_phecodes

In [ ]:
record_counts = rec

In [ ]:
run = wandb.init(project=project, job_type="log_artifact", entity=entity, tags=["artifact"])

In [ ]:
def get_path_from_wandb(reference: str):
    path = pathlib.Path(reference.split("file://")[1])
    assert path.exists()
    return path

def read_artifact(run, artifact_name, type_name):
    
    artifact = run.use_artifact(f'cardiors/recordgraphs/{artifact_name}:latest', type=type_name)
    file_path = get_path_from_wandb(artifact.manifest.entries[artifact_name].ref)
    print(file_path)

    return file_path

In [ ]:
def add_artifact(run, artifact_name, type_name, description, artifact_path, df):
   
    artifact = wandb.Artifact(artifact_name, type=type_name, 
                              description=description,
                             metadata = {"1_shape": f"{len(df)}x{len(df.columns)}",
                                         "2_cols": str(df.columns.to_list())})
    artifact.add_reference(f"""file://{artifact_path}""", artifact_name, checksum=True)
    run.log_artifact(artifact)

In [ ]:
phecode_defs_path = "/sc-projects/sc-proj-ukb-cvd/data/mapping/phecodes/phecode_strings_V2.csv"

artifact_name = "phecode_definitions"
type_name = "prepare_phecodes"
description = "Definition of phecodes"
artifact_path = f"{dataset_path}/artifacts/{artifact_name}_{artifact_date}.feather"

phecode_defs = pd.read_csv(phecode_defs_path, dtype={"phecode": str}).sort_values("phecode").reset_index()
#phecode_defs.to_feather(artifact_path)
#add_artifact(run, artifact_name, type_name, description, artifact_path, phecode_defs)

In [ ]:
phecode_mapping_path = "/sc-projects/sc-proj-ukb-cvd/data/mapping/phecodes/ICD10_to_phecode_V2.csv"

artifact_name = "icd10_phecode_mapping"
type_name = "prepare_phecodes"
description = "Mapping of ICD10 to phecodes"
artifact_path = f"{dataset_path}/artifacts/{artifact_name}_{artifact_date}.feather"

phecode_mapping = pd.read_csv(phecode_mapping_path, dtype={"icd10": str, "phecode": str})\
    .rename(columns={"icd10": "ICD10"})\
    .assign(ICD10 = lambda x: x.ICD10.str.replace(".", "", regex=False))\
    .reset_index(drop=True)
#phecode_mapping.to_feather(artifact_path)

#add_artifact(run, artifact_name, type_name, description, artifact_path, phecode_mapping)

## Prepare SNOMED -> ICD10 -> PheCode Mapping

In [ ]:
vocab_dir = f"{mapping_path}/athena"
concepts = pd.read_csv(f"{vocab_dir}/CONCEPT.csv", sep='\t')
concepts.concept_id = concepts.concept_id.apply(lambda s: f"OMOP_{s}")

In [ ]:
snomed_concepts = concepts.query('vocabulary_id == "SNOMED"').copy()
snomed_concepts.head()

In [ ]:
sct_icd = pd.read_csv(f"{mapping_path}/umls/der2_iisssccRefset_ExtendedMapFull_US1000124_20220301.txt", sep="\t")
sct_icd.head()

In [ ]:
mapping_df = sct_icd.merge(
    snomed_concepts, left_on="referencedComponentId", right_on="concept_code", how="left"
)

In [ ]:
mapping_df = mapping_df.merge(phecode_mapping, left_on="mapTarget", right_on="ICD10", how="left")
mapping_df.head()

In [ ]:
valid_rules = ("TRUE", "OTHERWISE TRUE", "IFA 248152002 | Female (finding) |", "IFA 248153007 | Male (finding) |")

mapping_df = mapping_df[mapping_df.mapRule.isin(valid_rules)]
mapping_df = mapping_df[
    ["concept_id", "phecode", "ICD10", "mapRule", "concept_code"]
].drop_duplicates()

mapping_df = mapping_df[~mapping_df.phecode.isna()]
mapping_df = mapping_df[~mapping_df.concept_id.isna()]
mapping_df.concept_id = mapping_df.concept_id.astype("category")
mapping_df.concept_id = mapping_df.concept_id.apply(lambda s: s.split("OMOP_")[1]).astype(int)
mapping_df.phecode = mapping_df.phecode.astype("category")
mapping_df.ICD10 = mapping_df.ICD10.astype("category")
mapping_df.mapRule = mapping_df.mapRule.astype("category")

mapping_df

In [ ]:
artifact_name = "snomed_icd10_phecode_mapping"
type_name = "prepare_phecodes"
description = "Mapping of SNOMED to phecodes via ICD10"
artifact_path = f"{dataset_path}/artifacts/{artifact_name}_{artifact_date}.feather"

mapping_df.reset_index(drop=True).to_feather(artifact_path)
add_artifact(run, artifact_name, type_name, description, artifact_path, phecode_mapping)

## Prepare Records TTE

In [ ]:
# load native file
records_raw_wo_death = pd.read_feather(read_artifact(run, "complete_records_extended", "prepare_records"))\
    .query("concept_id==concept_id")\
    .assign(concept_id = lambda x: x.concept_id.astype(int))
records_raw_wo_death.origin.unique()

In [ ]:
# prepare explicit coding of death
records_death = records_raw_wo_death.query('origin=="death_records"').copy()
records_death = records_death.drop_duplicates(subset=['eid', "death_date"], keep='first').copy()
records_death[["vocabulary_origin", "concept_id", "code", "concept_name", "domain_id", "concept_class_id", "vocabulary_id"]] = ["SNOMED", 4306655, '419620001', 'Death', 'Observation', 'Event', 'SNOMED']

In [ ]:
# concat to aggregated records + make sure we can trace back the mapped records
records_raw = pd.concat([records_raw_wo_death, records_death], axis=0)\
    .reset_index(drop=True)\
    .reset_index()\
    .rename(columns={"index":"unique_record_id"})

## Build phecode representation

### Separate SNOMED and ICD records

In [ ]:
records_raw.value_counts("origin")

In [ ]:
# identify original snomed records
records_raw_sct = (
    records_raw[records_raw.vocabulary_origin.isin(("CTV3", "SNOMED"))]
    .query('vocabulary_id == "SNOMED"')
    .copy()
)

records_raw_sct.concept_id = records_raw_sct.concept_id.astype(int)
records_raw_sct.value_counts("origin")

In [ ]:
# identify original ICD records
records_raw_icd = records_raw[records_raw.vocabulary_origin == "ICD10"].copy()
records_raw_icd.concept_id = records_raw_icd.concept_id.astype(int)
records_raw_icd.value_counts("origin")

In [ ]:
# check for remaining records
rest_idxs = records_raw_sct.index.to_list() + records_raw_icd.index.to_list()
records_rest = records_raw.loc[~records_raw.index.isin(rest_idxs)]
records_rest.value_counts("origin")

In [ ]:
records_rest.value_counts("vocabulary_origin")

In [ ]:
# check that its not coming from SNOMED or ICD codes
records_rest.query("vocabulary_origin=='SNOMED'").value_counts("vocabulary_id")

In [ ]:
records_raw_sct.shape, records_raw_icd.shape

In [ ]:
records_raw_icd

In [ ]:
records_raw_sct

### Map ICD records to phecodes

In [ ]:
icd_phecodes_1 = records_raw_icd.merge(phecode_mapping, left_on="code_origin", right_on="ICD10", how="left")

In [ ]:
icd_phecodes_1_mapped = icd_phecodes_1.query("phecode==phecode")

In [ ]:
icd_phecodes_1_unmapped = icd_phecodes_1.query("phecode!=phecode").drop(["phecode", "ICD10"], axis=1, inplace=False)

In [ ]:
# map unmapped to parent icd codes
icd_phecodes_2 = icd_phecodes_1_unmapped\
    .assign(code_origin_cut = lambda x: x.code_origin.str.slice(start=0, stop=3))\
    .merge(phecode_mapping, left_on="code_origin_cut", right_on="ICD10", how="left").drop(["code_origin_cut"], axis=1, inplace=False)

In [ ]:
icd_phecodes_2_mapped = icd_phecodes_2.query("phecode==phecode")

In [ ]:
# sanity checks of the missing maps
icd_phecodes_2_unmapped = icd_phecodes_2.query("phecode!=phecode")
icd_phecodes_2_unmapped_agg = icd_phecodes_2_unmapped.assign(code_origin_cut = lambda x: x.code_origin.str.slice(start=0, stop=3)).value_counts("code_origin_cut")
#print(icd_phecodes_2_unmapped_agg.to_string())

In [ ]:
# agg icd to phecode maps
icd_phecodes = pd.concat([icd_phecodes_1_mapped, icd_phecodes_2_mapped], axis=0)

In [ ]:
icd_phecodes

In [ ]:
icd_phecodes.concept_id = icd_phecodes.phecode.apply(lambda s: f"phecode_{s}")
icd_phecodes.concept_id = icd_phecodes.concept_id.astype("category")

icd_phecodes.vocabulary_id = "phecode"

icd_phecodes.drop(["phecode", "ICD10"], axis=1, inplace=True)
icd_phecodes

### Map SNOMED records to phecodes

In [ ]:
sct_phecodes_1 = records_raw_sct.merge(mapping_df[["concept_id", "phecode", "mapRule"]], on="concept_id", how="left") 
sct_phecodes_1

In [ ]:
sct_phecodes_1_mapped = sct_phecodes_1.query("phecode==phecode")

In [ ]:
sct_phecodes_1_mapped.shape

In [ ]:
sct_phecodes_1_mapped

In [ ]:
sct_phecodes_1_unmapped = sct_phecodes_1.query("phecode!=phecode")

In [ ]:
sct_phecodes_1_unmapped.shape

In [ ]:
sct_phecodes_1_unmapped.query("domain_id=='Condition'").value_counts("concept_name")

In [ ]:
# add and merge baseline covariates for merge rules
baseline_covariates = pd.read_feather(read_artifact(run, "baseline_covariates", "prepare_covariates"))
baseline_covariates.head()

In [ ]:
sct_phecodes = sct_phecodes_1_mapped.merge(baseline_covariates[['eid', 'sex_f31_0_0']], on='eid')
sct_phecodes

In [ ]:
sct_phecodes['sex_is_correct'] = True
false_indices = sct_phecodes.query('(mapRule == "IFA 248152002 | Female (finding) |") & (sex_f31_0_0 == "Male")').index
sct_phecodes.loc[false_indices, "sex_is_correct"] = False
false_indices = sct_phecodes.query('(mapRule == "IFA 248153007 | Male (finding) |") & (sex_f31_0_0 == "Female")').index
sct_phecodes.loc[false_indices, "sex_is_correct"] = False
sct_phecodes = sct_phecodes.query('sex_is_correct').copy()

In [ ]:
sct_phecodes.concept_id = sct_phecodes.phecode.apply(lambda s: f"phecode_{s}")
sct_phecodes.concept_id = sct_phecodes.concept_id.astype("category")

sct_phecodes["vocabulary_id"] = "phecode"

sct_phecodes.drop(["phecode", "mapRule", "sex_f31_0_0"], axis=1, inplace=True)

### Sanity checks

In [ ]:
phecode_records = pd.concat([sct_phecodes, icd_phecodes], axis=0)
phecode_records = phecode_records.query("date > recruitment_date")
phecode_records = phecode_records.sort_values(by="date")[["eid", "origin", "concept_id"]].drop_duplicates()

In [ ]:
phecode_records

In [ ]:
phecode_defs.phecode = phecode_defs.phecode.apply(lambda s: f"phecode_{s}")

In [ ]:
concept_counts = phecode_records.groupby(["origin", "concept_id"]).count()
concept_counts = concept_counts["eid"].sort_values(ascending=False).to_frame(name="count")

In [ ]:
concept_counts_merged = concept_counts.reset_index().merge(phecode_defs, left_on="concept_id", right_on="phecode", how='left')
concept_counts_merged = concept_counts_merged[['count', 'origin', 'concept_id', 'phecode_string', 'phecode_category']].query('count > 0')
concept_counts_merged[:20]

In [ ]:
concept_counts = phecode_records[['concept_id', 'eid']].groupby(["concept_id"]).count()
concept_counts = concept_counts["eid"].sort_values(ascending=False).to_frame(name="count")

In [ ]:
concept_counts_merged = concept_counts.reset_index().merge(phecode_defs, left_on="concept_id", right_on="phecode", how='left')
concept_counts_merged = concept_counts_merged[['count', 'concept_id', 'phecode_string', 'phecode_category']].query('count > 0')
concept_counts_merged[:20]

In [ ]:
concept_counts_merged[~concept_counts_merged.phecode_category.isin(("Cong", "Dev", "Neonate"))].query(
    "count >= 25"
)

In [ ]:
#concept_counts_merged.reset_index(drop=True).to_feather("/home/USER/concept_counts_snomed_220527.feather")
#concept_counts_merged.reset_index(drop=True).to_excel("/home/USER/concept_counts_snomed_220527.xlsx")

In [ ]:
run.log(dict(incident_phecode_counts=wandb.Table(dataframe=concept_counts.reset_index())))

## Concatenate records

In [ ]:
records_raw.concept_id = records_raw.concept_id.apply(lambda s: f'OMOP_{s}')

In [ ]:
records_raw

In [ ]:
records_all_vocabs = pd.concat([
    records_raw,#.assign(concept_id = lambda x: f'OMOP_{x.concept_id}'), 
    icd_phecodes, 
    sct_phecodes], axis=0).rename(dict(vocabulary_id='vocabulary'), axis=1).reset_index(drop=True)
records_all_vocabs

In [ ]:
use_columns = [
    'eid',
    'birth_date',
    'recruitment_date',
    'death_date',
    'cens_date',
    'exit_date',
    'domain_id',
    'code',
    'concept_id',
    "unique_record_id",
    'origin',
    'date',
    'vocabulary',    
]

records_all_vocabs = records_all_vocabs[use_columns].copy()

In [ ]:
records_all_vocabs.info()

In [ ]:
# make everything faster
for col in tqdm(records_all_vocabs.columns): 
    if "date" in col: records_all_vocabs[col] = records_all_vocabs[col].astype("datetime64[D]")

In [ ]:
# make everything faster
for col in tqdm(["concept_id", "vocabulary"]): 
    records_all_vocabs[col] = records_all_vocabs[col].astype("category")

In [ ]:
records_mapped_unique = records_all_vocabs.drop_duplicates().reset_index(drop=True)

In [ ]:
records_mapped_unique.shape

In [ ]:
records_mapped_unique.info()

In [ ]:
def add_artifact(run, artifact_name, type_name, description, artifact_path, df):
   
    artifact = wandb.Artifact(artifact_name, type=type_name, 
                              description=description,
                             metadata = {"1_shape": f"{len(df)}x{len(df.columns)}",
                                         "2_cols": str(df.columns.to_list())})
    artifact.add_reference(f"""file://{artifact_path}""", artifact_name, checksum=True)
    run.log_artifact(artifact)

In [ ]:
# log in wandb
artifact_name = "final_records_omop"
type_name = "prepare_records"
description = "Cleaned first patient records from the data portal in long format"
artifact_path = f"{dataset_path}/artifacts/{artifact_name}_{artifact_date}.feather"
records_mapped_unique.to_feather(artifact_path)
add_artifact(run, artifact_name, type_name, description, artifact_path, records_mapped_unique)

In [ ]:
1+1